# Custom Signals Example

This notebook shows how to construct and backtest a custom signal DataFrame.

In [ ]:
import pandas as pd
from bist_quant import DataLoader, PortfolioEngine

loader = DataLoader()
prices_raw = loader.load_prices()

In [ ]:
if {"Date", "Ticker", "Close"}.issubset(prices_raw.columns):
    prices = prices_raw.pivot(index="Date", columns="Ticker", values="Close").sort_index()
else:
    prices = prices_raw.copy()

# Example custom signal: 20-day momentum rank centered at 0
momentum = prices.pct_change(20)
custom_signal = momentum.rank(axis=1, pct=True) - 0.5
custom_signal = custom_signal.fillna(0.0)

custom_signal.tail()

In [ ]:
engine = PortfolioEngine(options={"top_n": 15, "rebalance_frequency": "monthly"})

result = engine.run_backtest(
    signals=custom_signal,
    start_date="2023-01-01",
    end_date="2023-12-31",
)

result.metrics

In [ ]:
result.returns.cumsum().plot(figsize=(12, 5), title="Custom Signal Cumulative Returns")